In [1]:
# from deep_ensemble_keras import create_model
import os,sys
import json
import ember
import csv
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn import preprocessing
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input, Dense, Dropout
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical
from keras.models import load_model
from tensorflow import keras
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp
from scipy.spatial import distance
from sklearn.decomposition import PCA

%matplotlib inline

from collections import OrderedDict
from reliability_diagrams import *

Using TensorFlow backend.


In [2]:
def create_model(dim_input = 2381, dim_output = 2, lr = 0.01):
#     print("Building neural network")
    model = Sequential()
    model.add(Dense(70, input_shape=(dim_input,), activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(70, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(dim_output, activation='softmax'))

#     print("Compiling neural network")
    # compile the model
    opt = keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def extract_data():
    data_dir_2017 = 'new_ember_2017_2'

    data_dir_2018 = 'new_ember2018'

    X_test_2017, y_test_2017 = ember.read_vectorized_features(data_dir_2017, subset = 'test', feature_version=2)

    X_test_2018, y_test_2018 = ember.read_vectorized_features(data_dir_2018, subset = 'test', feature_version=2)

    return X_test_2017, y_test_2017, X_test_2018, y_test_2018

def accuracy(y_hat, y):
    pred = np.argmax(y_hat, 1)
    return (pred == y).mean()

def get_result(y_hat):
    return np.argmax(y_hat, 1)

In [3]:
def get_accuracy_for_deep_ensemble(X_test_2017, y_test_2017, X_test_2018, y_test_2018):
    
    acc_2017, acc_2018 = [], []
    
    for i in range(5):
        
        print(i)
        
        model = create_model()

        model.load_weights("models_keras/model_{}.h5".format(i))
        
        acc_2017.append(accuracy(model.predict(X_test_2017), y_test_2017))
        
        acc_2018.append(accuracy(model.predict(X_test_2018), y_test_2018))
        
    return acc_2017, acc_2018

In [4]:
X_test_2017, y_test_2017, X_test_2018, y_test_2018 = extract_data()

In [5]:
X_test_2017 = preprocessing.normalize(X_test_2017, norm='l2', axis = 0)
X_test_2018 = preprocessing.normalize(X_test_2018, norm='l2', axis = 0)

In [7]:
pca = PCA(n_components=500)
X_test_2017_pca = pca.fit_transform(X_test_2017)
X_test_2018_pca = pca.fit_transform(X_test_2018)

In [6]:
acc_2017, acc_2018 = get_accuracy_for_deep_ensemble(X_test_2017, y_test_2017, X_test_2018, y_test_2018)

0
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
1
2
3
4


In [7]:
acc_2017

[0.875135, 0.88449, 0.9446, 0.880875, 0.961215]

In [8]:
acc_2018

[0.80637, 0.81192, 0.81482, 0.83674, 0.8376]

In [8]:
# acc_2018

In [9]:
# 95% n = 5: 0.56327

t_val, p_val = ks_2samp(acc_2017, acc_2018, mode = 'asymp')

In [10]:
t_val,p_val

(1.0, 0.013475889875863678)

In [3]:
def ks_test(dataset_one, dataset_two, sig_value):
    p_vals = []

    # For each dimension we conduct a separate KS test
    for i in range(dataset_one.shape[1]):
        feature_tr = dataset_one[:, i]
        feature_te = dataset_two[:, i]

        t_val, p_val = ks_2samp(feature_tr, feature_te)

        p_vals.append(p_val)

    # Apply the Bonferroni correction to bound the family-wise error rate. This can be done by picking the minimum
    # p-value from all individual tests.
    p_vals = np.array(p_vals)
    p_val = min(np.min(p_vals), 1.0)

    return p_val, sig_value/dataset_one.shape[1], p_vals

In [12]:
# observed_p, alpha, p_vals = ks_test(X_test_2017,X_test_2018, 0.05)

In [13]:
# observed_p, alpha

In [13]:
class MMDStatistic:
    r"""The *unbiased* MMD test of :cite:`gretton2012kernel`.
    The kernel used is equal to:
    .. math ::
        k(x, x') = \sum_{j=1}^k e^{-\alpha_j\|x - x'\|^2},
    for the :math:`\alpha_j` proved in :py:meth:`~.MMDStatistic.__call__`.
    Arguments
    ---------
    n_1: int
        The number of points in the first sample.
    n_2: int
        The number of points in the second sample."""

    def __init__(self, n_1, n_2):
        
        self.n_1 = n_1
        self.n_2 = n_2
        
        # The three constants used in the test.
        self.a00 = 1. / (n_1 * (n_1 - 1))
        self.a11 = 1. / (n_2 * (n_2 - 1))
        self.a01 = - 1. / (n_1 * n_2)

    def __call__(self, sample_1, sample_2, alphas, ret_matrix=False):
        r"""Evaluate the statistic.
        The kernel used is
        .. math::
            k(x, x') = \sum_{j=1}^k e^{-\alpha_j \|x - x'\|^2},
        for the provided ``alphas``.
        Arguments
        ---------
        sample_1: :class:`torch:torch.autograd.Variable`
            The first sample, of size ``(n_1, d)``.
        sample_2: variable of shape (n_2, d)
            The second sample, of size ``(n_2, d)``.
        alphas : list of :class:`float`
            The kernel parameters.
        ret_matrix: bool
            If set, the call with also return a second variable.
            This variable can be then used to compute a p-value using
            :py:meth:`~.MMDStatistic.pval`.
        Returns
        -------
        :class:`float`
            The test statistic.
        :class:`torch:torch.autograd.Variable`
            Returned only if ``ret_matrix`` was set to true."""
        sample_12 = torch.cat((sample_1, sample_2), 0)
        distances = pdist(sample_12, sample_12, norm=2)

        kernels = None
        for alpha in alphas:
            kernels_a = torch.exp(- alpha * distances ** 2)
            if kernels is None:
                kernels = kernels_a
            else:
                kernels = kernels + kernels_a

        k_1 = kernels[:self.n_1, :self.n_1]
        k_2 = kernels[self.n_1:, self.n_1:]
        k_12 = kernels[:self.n_1, self.n_1:]

        mmd = (2 * self.a01 * k_12.sum() +
               self.a00 * (k_1.sum() - torch.trace(k_1)) +
               self.a11 * (k_2.sum() - torch.trace(k_2)))
        if ret_matrix:
            return mmd, kernels
        else:
            return mmd

    def pval(self, distances, n_permutations=1000):
        r"""Compute a p-value using a permutation test.
        Arguments
        ---------
        matrix: :class:`torch:torch.autograd.Variable`
            The matrix computed using :py:meth:`~.MMDStatistic.__call__`.
        n_permutations: int
            The number of random draws from the permutation null.
        Returns
        -------
        float
            The estimated p-value."""
        if isinstance(distances, Variable):
            distances = distances.data
        return permutation_test_mat(distances.cpu().numpy(),
                                    self.n_1, self.n_2,
                                    n_permutations,
                                    a00=self.a00, a11=self.a11, a01=self.a01)

In [14]:
def MMD(dataset_one, dataset_two):

    mmd_test = MMDStatistic(len(dataset_one), len(dataset_two))


    all_dist = distance.cdist(dataset_one, dataset_two, 'euclidean')
    
    
    median_dist = np.median(all_dist)

    # Calculate MMD.
    t_val, matrix = mmd_test(torch.autograd.Variable(torch.tensor(dataset_one)),
                             torch.autograd.Variable(torch.tensor(dataset_two)),
                             alphas=[1/median_dist], ret_matrix=True)
    p_val = mmd_test.pval(matrix)


    return p_val, np.array([])

In [15]:
MMD(X_test_2017_pca,X_test_2018_pca)

MemoryError: Unable to allocate 298. GiB for an array with shape (200000, 200000) and data type float64